[Youtube Link](https://www.youtube.com/watch?v=F6u5rhUQ6dU) | [Slides](https://www.slideshare.net/nnja/memory-management-in-python-the-basics) | **Date:** March 1 2020

* Ability to write more efficient programs
* Better debugging with lower performing programs

# Variable
* Python has names (not variables)
* We have names -> references -> objects
* Name is just a label for an object; in python, each object can have lots of names

## Different types of objects
**Simple**
* numbers
* strings

**Containers**
* dict
* list
* user defined classes
* These containers store refs to either simple objects or other containers

## What is a reference?
* Name or container object that points at another object

### Reference Count?
* Number of references that you have to any object
* We can increase the reference count by doing something as simple as `x=300`, `y=300`, the object `300` has 2 references
* We can decrease the ref count using `del`, e.g. `del x`
* What does `del` do?
    * Does not delete the object
    * removes that name a sa reference to that object
    * Hence reduces the ref count
* We can also change reference count by setting it to something directly, e.g. `y = None`, so we deleted the reference `x` by using `del x` and then set `y` to `None`, now there are no more references to the object `300`
* Now, zero references to object `300` hence we don't care if it exists anymore, hence it can be safely removed from the memory later one
* We can also decreate ref count by going out of scope

```
def print_word():
    word = 'Seven' # reference count of str object 'Seven' = 1
    print('word is', word)

print_word()

# at this stage, ref count is decreased to zero since we are no longer in scope of function `print_word`
```
* Finally, when a program exits, the reference count is decreased too since we simply have no more references
* **local vs global namespace**
    - Global namespace objects might never go out of scope and their ref count never reaches 0
    - Avoid putting large/complex objects in global namespace
    - So, avoid putting objects in global namespace and if you do, set their reference count to 0 by calling `del` on them

### Every python object holds three things
* Its Type
* Its Value
* Its Reference Count

# What is Garbage Collection?

* A way for python to automatically release memory when the object that is using that memory is no longer in use.

## Two main types of GC

* Reference Counting
* Tracing
**Python uses both**
* How does reference counting gc work?
    - Add/remove references, ref count ++ on assignment and -- on `del` or any of the other operations that reduce ref count
    - When ref count reaches 0, the object is deleted
        - Cascading effect - Other objects that the deleted object pointed to, if that object's ref count reaches 0, those objects are deleted too

**Good**
- Easy to implement
- When refcount == 0, objects are immediately deleted

**Bad**
- Space overhead, ref count for every object takes some data space
- Execution overhead, ref count changes on every assignment

**Ugly**
- Not generally thread safe 
- Reference counting garbage collection doesn't detect cyclical references

### Cyclical reference by example

```
class Node:
    def __init__(self, value):
        self.value = value
    
    def next(self, next)
        self.next = next)
```
- The node class contains a value and a reference to the next node.

```
# Let us declare a root, left and right nodes
root = Node('root')
left = Node('left')
right = Node('right')

root.next(left)
left.next(right)
right.next(left)

# Ref cout of root: 1 (referred by `root`)
# Ref count of left: 3 (referred by `left`, `root` and `right`)
# Ref count of right: 2 (referred by `right`, `left`)
```
- Now as we `del` the names;

```
del root
del left
del right
```
- Althought we remove the names, the internal references (`.next`) is still there
- So the ref count of `root` is 0, by ref count of `left` and `right` (referred by `right.next` and `left.next` respectively) remains 1
- **Ref counting alone will not gc objects with cyclical references**

### Tracing Garbage Collection
- So let us look at another type of gc in Python, called **Tracing**
- Algo used: **Mark and Sweep**
    -  Start from root node and mark any objects whose references are found _(this algo is run when references reach a certain threshold)_
    - Next: Sweep - When marking is done, **Sweep** phase will remove the dead objects
    - This includes deletion of cyclical references too

### What does Python use?
- It uses **Reference Counting** and also a strategy called **Generational**
- **Generational** is a type of **Tracing Garbage Collection**

#### Generational Garbage Collection
- Based on theory **most objects die young**
- Example:
    - Frequently objects are created, often temporary which are used once or a few times and then never used again
- Python maintains list of every object created as a program is run
- It actually makes three lists:
    - generation 0
    - generation 1
    - generation 2
- Newly created objects are stored in _generation 0_
- Each object is stored in only one generation and we optimize by collecting objects in _generation 0_ more frequently than the other generations
- **Remember**: Only container objects with a refcount > 0 are stored in a generation list.
- So similar to Mark and Sweep, but we only keep the ones with active references - this means, fewer objects are tracked and scanned.
- When number of objects in a generation reaches a threshold, python runs gc algo on that generation, and on any generations younger than it.
    - So if we run on generation 2, we are actually collecting on generation 0, 1 and 2.

**What happens during a generational garbage collection cycle?**
    - Python makes a list of objects to discard
    - Runs an algorithm to detect reference cycles
    - If an object has no outside references, it's put on the discard list
    - When the cycle is done, it frees up the objects on the discard list

- After gc cycle, objects that survived are promoted to the next generation
- Objects in the last generation (generation 2) stay there as the program executes.

**Big Idea:**
    - When ref count reaches 0, we get immediate clean up
    - But if we have objects with cyclical references, we will need for gc to run, and in order for that to happen, we will have to wait.
    - So if we have cyclical refs, it can potentially slow down our progam.
    
### Reference Counting Gotchas
- Not generally thread safe
    - Think what happens if two threads try to increase and decrease ref count at the same time
- Remember cylical refs `left.next` and `right.next`? They will be cleaned up by generational gc **EXCEPT in python2** if they have a `__del__` method.
    - This is fixed as of Python 3.4
- What is `__del__` magic method?
    - Sometimes called a `destructor`
    - **NOT** the `del` statement - this is not invoked when `del` statement is called
    - This is only called when the reference count for an object reaches 0
    - So this is ran before an object is removed from memory.

### Strategy to improve mem usage: `__slots__`
#### What are `__slots__`?**
- Python class instances contain dictionary of its names and values, e.g.

```
class Dog(object):
    pass

buddy = Dog()
buddy.name = 'Buddy'

print(buddy.__dict__)
{'name': 'Buddy'}
```
- `__slots__` turns the internal dictionary to tuples, tuples are immutables.
- `__slots__` prevents us from setting attributes on instances
- So for example, I can use `__slots__` to prevent setting unnecessary attributes on a class, e.g.

```
class Point:
    __slots__ = ('x', 'y')

point = Point()
point.x = 5
point.y = 7

point.name = "something" # Raises AttributeError
```

In [5]:
class Point:
    __slots__ = ('x', 'y')
    
point = Point()
point.x = 5
point.y = 5

point.name = "A name"

AttributeError: 'Point' object has no attribute 'name'

In [6]:
point.__dict__ # My point doesn't have an internal dictionary

AttributeError: 'Point' object has no attribute '__dict__'

In [7]:
# Let us define the PointTwo class without slots and then try to access its instance's dictionary
class PointTwo:
    pass

point = PointTwo()
point.x = 5
point.y = 5
point.name = "Some Name" # Now this should not raise Attribute Error
point.__dict__ # this should work and 'x', 'y', 'name' along with their values should appear

{'x': 5, 'y': 5, 'name': 'Some Name'}

<!-- Continuing points under __slots__ heading -->
#### So why are `__slots__` important?
**size of `dict` vs size of `tuple`**

```
import sys
sys.getsizeof(dict())
sys.getsizeof(tuple())

```
- In my installation:
    - dict: 232 bytes
    - tuple: 40 bytes

#### When would we wanna use `__slots__`?
- If we're going to create lots of instances of a class
- If we know in advance what properties the class should have


# What is GIL

**GIL** is Global Interpreter Lock
- Prevents multiple threads from executing python code at the same time
- One **GIL** for each python interpreter
- In other words two python programs running on one machine don't share a **GIL**
- Only one thread can run in the interpreter at a time

**Why does python need interpreter lock?**
- We need to prevent reference counts from being changed concurrently

**Advantages of GIL**
- Upside
    - Fast & Simple GC
- Downside
    - In a python program, no matter how many threads exist, only one thread will be executed at a time

**Want to take advantage of multiple CPUs?**
- Use multi-processing instead of multi-threading.
- Each process will have its **own** GIL. It is on the developer to figure out a way to share information between processes.

## If the GIL limits us, can't we remove it?
- Lots of discussions, a patch was submitted once to remove it, it never went through - various contentions
- For better or worse, **GIL** is here to stay
- Some Python implementations, e.g. IronPython, Jython do not have **GIL**

### Final points
- Consider python3 - it has better GIL implementation
- Consider numpy/scipy for scientific/number intensive applications as they are more efficient for such purposes
- Be mindful of these learnings as they will help write more efficient code.